In [1]:
import pandas as pd
data = pd.read_csv('./data00.csv')
data.head()

,名称,经度,纬度,距离,建筑年代,楼栋总数,房屋总数
0,正阳世纪星城(绅鸿苑),121.480515,30.912384,880,2005.0,29,338
1,奉苑新村,121.466207,30.920155,1945,1999.0,26,274
2,南亭公寓,121.460741,30.907867,2723,2000.0,25,266
3,九华丽苑,121.460980,30.925206,2549,2014.0,20,741
4,曙光新村(南桥),121.472526,30.915556,1369,1993.0,28,277


In [2]:
import folium
import math
# 0 . 指定地图显示中心，即本购物中心的经纬度
# 由于该点的坐标由高德API获得，所以是火星坐标系
# folium的格式于高德相反，是：纬度在前，经度在后
map_center = [30.918412,121.486477]

# 1. 生成底图
# folium默认采用openstreet风格的地图，采用WGS84坐标系
# 可通过tiles来换成高德瓦片底图，规避坐标系不同的问题
# 若使用mapbox底图，需要token；没有token的最多zoom到2
# ref: http://www.mamicode.com/info-detail-2941961.html
m = folium.Map(location = map_center,
               zoom_start = 13.4,
               tiles = 'http://webst01.is.autonavi.com/appmaptile?style=7&x={x}&y={y}&z={z}',
               attr = '© <a href="http://ditu.amap.com/">高德地图</a>',   #用于给自定义的tiles命名
               control_scale = True #添加比例尺
              )
# Important: 只有按照先添加Circle，然后添加CircleMarker，最后添加中心点Marker的顺序
# 才能全部显示出来，不然就只能显示CircleMarker或者只显示Marker。原因未知。

# 2. 绘制3Km半径圆
# Cicle函数和CicleMarker分别以米和像素为单位
# 经测试Cicle和CilceMarker可以一起显示,但不能与Marker一起显示
folium.Circle(
    radius = 3000,
    location = map_center,
    color = 'crimson',
    fill = False,
).add_to(m)

# 3. 绘制气泡图Bubble Map
# 通过自定义CicleMarker的半径(以像素为单位)来实现
for i in range(len(data)):
    location = [data['纬度'][i],data['经度'][i]]
    folium.CircleMarker(location,
                        radius=math.ceil(data['房屋总数'][i]/800)*3,
                        weight = 0, #设置线宽为0，
                        #color='yellow',线宽不为零的时候将显示
                        fill=True,
                        fill_color='blue',
                        fill_opacity=0.6
                       ).add_to(m)

# 4. 绘制中心点pop-up marker
folium.Marker(
    location = map_center,
    popup = 'The mall',
    icon = folium.Icon(icon='star',color='red')
).add_to(m) 

m